In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-02-24 04:17:26.111320


In [4]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [5]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [6]:
current_date.to_datetime64()

numpy.datetime64('2025-02-24T04:00:00.000000000')

In [7]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [8]:
fetch_data_to

Timestamp('2025-02-24 04:00:00+0000', tz='UTC')

In [9]:
fetch_data_from

Timestamp('2025-01-26 04:00:00+0000', tz='UTC')

In [10]:
from datetime import datetime, timedelta
import pandas as pd
from src.data_utils import load_and_process_taxi_data
import pytz  # Import timezone support

def fetch_batch_raw_data(from_date, to_date):
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Convert to timezone-aware datetime (assuming UTC)
    timezone = pytz.UTC  # Change this to your actual timezone if needed
    from_date = from_date.replace(tzinfo=timezone)
    to_date = to_date.replace(tzinfo=timezone)

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    
    # Ensure `pickup_datetime` is also timezone-aware
    rides_from["pickup_datetime"] = pd.to_datetime(rides_from["pickup_datetime"], utc=True)
    
    rides_from = rides_from[rides_from["pickup_datetime"] >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to["pickup_datetime"] = pd.to_datetime(rides_to["pickup_datetime"], utc=True)
        rides_to = rides_to[rides_to["pickup_datetime"] < historical_to_date]

        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Shift the data forward by 52 weeks
    rides["pickup_datetime"] += timedelta(weeks=52)

    # Sort for consistency
    rides.sort_values(by=["pickup_location_id", "pickup_datetime"], inplace=True)

    return rides


In [11]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-01.
Loading data for 2024-01...
Total records: 2,964,624
Valid records: 2,911,483
Records dropped: 53,141 (1.79%)
Successfully processed data for 2024-01.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!


In [12]:
rides

,pickup_datetime,pickup_location_id
1640658,2025-02-12 16:25:44+00:00,2
1947359,2025-02-15 16:56:40+00:00,2
342868,2025-01-26 06:32:18+00:00,3
343895,2025-01-26 12:56:32+00:00,3
70385,2025-01-27 03:59:58+00:00,3
...,...,...
2726606,2025-02-24 02:37:37+00:00,263
2726770,2025-02-24 03:03:21+00:00,263
2872560,2025-02-24 03:38:20+00:00,263
2726810,2025-02-24 03:48:48+00:00,263


In [13]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [14]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-26 04:00:00+00:00,2,0
1,2025-01-26 05:00:00+00:00,2,0
2,2025-01-26 06:00:00+00:00,2,0
3,2025-01-26 07:00:00+00:00,2,0
4,2025-01-26 08:00:00+00:00,2,0
...,...,...,...
174691,2025-02-23 23:00:00+00:00,263,17
174692,2025-02-24 00:00:00+00:00,263,13
174693,2025-02-24 01:00:00+00:00,263,11
174694,2025-02-24 02:00:00+00:00,263,4


In [15]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   pickup_hour         174696 non-null  datetime64[ns, UTC]
 1   pickup_location_id  174696 non-null  int16              
 2   rides               174696 non-null  int16              
dtypes: datetime64[ns, UTC](1), int16(2)
memory usage: 2.0 MB


In [16]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-02-23 23:17:30,292 INFO: Initializing external client
2025-02-23 23:17:30,293 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-23 23:17:30,993 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214632


In [17]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 174696/174696 | Elapsed Time: 00:11 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214632/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)